# Queue systems simulation

## Le package SimPy 

### Pour installer le package simpy , utiliser la commande  : pip install simpy

SimPy est un package de simulation d'événement discret pour Python, il fait référence à un système de simulation qui introduira périodiquement un événement spécifié au fur et à mesure de son exécution. 

Dans SimPy, le système DES est contenu dans la classe Environnement  " Environment() "  . L'utilisateur spécifie un système qui fonctionnera pour l'Environnent lors de l'exécution Environment.run(). 

Au sein de l'Environnent, l'utilisateur peut définir les objets 'Process' et les 'Resource'  En particulier,


#### l'objet Process  
- Le générateur des événements discrets avec lesquels nous allons travailler. 

#### l'objet Ressource
- Resource définit un objet avec lequel interagir dans cet Environnement discret


## Les modules CLIENT et SOURCE 

Dans cette partie on définit deux modules
- Le module Source définit la source des clients , la durée inters arrivées suit une loi exponentielle de paramètre Lambda
- Le module Client définit le traitement de client lors de son arrivée ( mise en file , durée de service ) 


Voici les principales instructions dans le programme qui suit 
- on enregistre son temps d'arrivée 
- on vérifie si la file est vide, le client passe directement sinon il attend son tour dans la file 
- lorsqu'il sort de la file d'attente, on affiche le temps qu'il passé dans la file 
- le client passe une durée de son service suivant une loi exponentielle de paramétre U 
- lorsque le cleint est servi, il quitte le système et affiche son temps de sortie et la durée de son service 



Pour générer des valeurs qui suivent une loi expo, on peut utiliser la fonction random.expovariate() dans python directement , et on peut aussi utiliser la fonction suivante qui implémente la méthode de l'inverse : 


In [103]:
import math
import random

def generate_expo(λ):
    x = 0
    while x == 0:
        x = random.random()
    y = - math.log(1 - x) /λ
    return y

print(generate_expo(2) )    # exemple 

2.6110719086100143


In [104]:
from simpy import *
import random

#-----------------------------------------------------------------------------------------------------------------------------


def Client(env, name, Resource , L):
    
    # l'heure actuelle , enregistrer l'heure d'arrivée du client 
    arrive = env.now  
    
    sysLength = [ max([0, len(Resource.put_queue) + len(Resource.users)]) ]     # la taille de la file d'attente
    Qlength = [len(Resource.put_queue) ]

    # si le sytéme a une taille limité et la file est pleine 
    if  ( sysLength[0]>L and L!= -1 ):   
        yield env.timeout(0)  # passer à l'event suivant 
        sysLength[0] = sysLength[0] - 1 
        print("Heure : %7.3f  ||  %s: ----------------Refusé , le système est complèt , le nombre des cleints dans le système = %s " % (env.now, name ,  sysLength))
    else :
        print("Heure : %7.3f  ||  %s: Arrivé ||  nombre des clients dans le système= %s ||  nombre des clients dans la file= %s" % (env.now, name,sysLength, Qlength))
        with Resource.request() as req:  # demander le service 
            yield req   # attendre son tour 
            # le tepmps qu'il a attendu pour prendre sa demande 
            wait = env.now - arrive  
        
            # l'attente est terminé , le client se serve 
            print('Heure : %7.3f  ||  %s: Libéré de la file , durée attente dans la file = %6.3f ' % (env.now, name, wait))

            # génerer une durée de serive aléatoirement suivant une loi expo de paramétre --> U=1/timeServ
            dureeServ = random.expovariate(1.0 / timeServ)  

            # attendre que le service termine 
            yield env.timeout(dureeServ)

            # le service est terminé 
            print('Heure : %7.3f  ||  %s: Terminé , durée de service = %7.4f '  % (env.now, name , dureeServ))

#-----------------------------------------------------------------------------------------------------------------------------

def Source(env, number, arrivalMean,Resource , L):
    for i in range(number):   # génerer un nombre des clients = number  
        c = Client(env, 'Client%02d' % (i+1), Resource , L)   # définir le cleint et associer un nom 
        env.process(c)   # générer le client c 
        
        # générer une durée d'inter arrivée suivant une lois expo
        t = random.expovariate(1.0 / arrivalMean)     
        
        #attendre une durée t avant d arriver 
        yield env.timeout(t)   


# Le Systéme M/M/1

- la Durée inters arrivés suit une lois expo de paramètre lmd >0
- la Durée de service suit une lois expo de paramètre U >0
- la source des clients est infinie 
- la taille de système est infinie 
- le nombre des serveurs est 1
- la discipline de service est FIFO


In [105]:
maxClient = 100          # le nombre max des cleints dans le système 
maxTime = 5.0            # Limite de temps de simulation 
timeServ = 1/7.5         # Moyenne de durée de service 
arrivalMean = 1/5        # Moyenne de durée inter arrivés 
seed = 12345             # Seed pour la simulation 
L = -1                   # taille max = -1 pour dire qu'elle est infinie 


print ( "\nles caractéristiques de performance:\n ")

lmd = 1 / arrivalMean   # lambda taux d'arrivée des cleints 
U = 1 / timeServ        # U taux de départ 
P = lmd / U             # l'intensité de traffic 
ns= P / (1-P)           # le nombre moyen des clients dans le système 
nf = P*P  / (1-P)       # le nombre moyen des clients dans la file
ts = ns/lmd             # Temps de séjour moyen d’un client dans le système 
tf = nf/lmd             # Temps de séjour moyen d’un client dans la file 


print("Intensité du traffic : %6.4f " %(P))
print("Nombre moyen de clients dans le système ns = %6.4f " %(ns))
print("Nombre moyen de clients dans la file nf = %6.4f " %(nf))
print("Temps de sejour moyen d’un client dans le systeme ts =  %6.4f " %(ts))
print("Temps de séjour moyen d’un client dans la file tf = %6.4f  \n" %(tf))
print("\nSimulation de système M/M/1  ****************************************************************************************\n")

# configurer et commencer la simulation 
random.seed(seed)                                                         # initialiser le seed 
env = Environment()                                                       # préparer l'environnement 
env.process(Source(env, maxClient, arrivalMean, Resource(env,1) , L))     # intialiser le process générateur avec un seul serveur 
env.run(until=maxTime)                                                    # commencer la simulation avec un max de temps = maxTime




les caractéristiques de performance:
 
Intensité du traffic : 0.6667 
Nombre moyen de clients dans le système ns = 2.0000 
Nombre moyen de clients dans la file nf = 1.3333 
Temps de sejour moyen d’un client dans le systeme ts =  0.4000 
Temps de séjour moyen d’un client dans la file tf = 0.2667  


Simulation de système M/M/1  ****************************************************************************************

Heure :   0.000  ||  Client01: Arrivé ||  nombre des clients dans le système= [0] ||  nombre des clients dans la file= [0]
Heure :   0.000  ||  Client01: Libéré de la file , durée attente dans la file =  0.000 
Heure :   0.001  ||  Client01: Terminé , durée de service =  0.0014 
Heure :   0.108  ||  Client02: Arrivé ||  nombre des clients dans le système= [0] ||  nombre des clients dans la file= [0]
Heure :   0.108  ||  Client02: Libéré de la file , durée attente dans la file =  0.000 
Heure :   0.155  ||  Client02: Terminé , durée de service =  0.0473 
Heure :   0.457  || 

# Le Systéme M/M/s

- la Durée inters arrivés suit une lois expo de paramètre lmd >0
- la Durée de service suit une lois expo de paramètre U >0
- la source des clients est infinie 
- la taille de système est infinie 
- le nombre des serveurs est s
- la discipline de service est FIFO


In [106]:
import math
maxClient= 100          # le nombre max des cleints pour la simulation 
maxTime = 4             # Limite de temps de simulation 
timeServ = 1/6          # Moyenne de durée de service 
arrivalMean = 1/9       # Moyenne de durée inter arrivés 
seed = 12345            # Seed pour la simulation 
s= 2                    # nombre des serveurs 
k=0                     # variable pour le calcul de la série de P0
L = -1                  # taille max = -1 pour dire qu'elle est infinie 


print ( "\nles caractéristiques de performance:\n ")

lmd = 1 / arrivalMean                                            #lambda le paramétre de la loi expo de durée inter arrivés 
U = 1 / timeServ                                                 #U le paramétre de la loi expo de durée de service 
P = lmd / (s*U)                                                  # l'intensité de traffic 
somme = 0 
for k in range(s) : 
    somme = somme+  ((1/math.factorial(k)) * ((lmd/U)**k ) ) 

P0 = 1/ ( somme + ((1/math.factorial(s)) * ((lmd/U)**s) * (s*U/(s*U-lmd))) )  
Ps = (1/math.factorial(s) ) * ((lmd/U ) ** s ) * P0
nf = (P*Ps) / (1-P)**2
ns = nf + s*P
ts = ns/lmd
tf = nf/lmd
Pw = Ps/ (1-P)

print("Intensité du traffic : %6.4f " %(P))
print("La probabilité d'avoir 0 clients dans le système P0 =  %6.4f" % ( P0))
print("La probabilité d'avoir s cleints dans le système Ps =  %6.4f" % ( Ps))
print("La probabilité d'attente = %6.4f " %(Pw))
print("Nombre moyen de clients dans le système ns = %6.4f " %(ns))
print("Nombre moyen de clients dans la file nf = %6.4f " %(nf))
print("Temps de sejour moyen d’un client dans le systeme ts = %6.4f " %(ts))
print("Temps de séjour moyen d’un client dans la file tf = %6.4f  \n" %(tf))


# Setup and start the simulation
print("\nSimulation de système M/M/S  ******************************************************************************************\n")
random.seed(seed)
env = Environment()
env.process(Source(env, maxClient, arrivalMean, Resource(env,s) , L))
env.run(until=maxTime)



les caractéristiques de performance:
 
Intensité du traffic : 0.7500 
La probabilité d'avoir 0 clients dans le système P0 =  0.1429
La probabilité d'avoir s cleints dans le système Ps =  0.1607
La probabilité d'attente = 0.6429 
Nombre moyen de clients dans le système ns = 3.4286 
Nombre moyen de clients dans la file nf = 1.9286 
Temps de sejour moyen d’un client dans le systeme ts = 0.3810 
Temps de séjour moyen d’un client dans la file tf = 0.2143  


Simulation de système M/M/S  ******************************************************************************************

Heure :   0.000  ||  Client01: Arrivé ||  nombre des clients dans le système= [0] ||  nombre des clients dans la file= [0]
Heure :   0.000  ||  Client01: Libéré de la file , durée attente dans la file =  0.000 
Heure :   0.002  ||  Client01: Terminé , durée de service =  0.0017 
Heure :   0.060  ||  Client02: Arrivé ||  nombre des clients dans le système= [0] ||  nombre des clients dans la file= [0]
Heure :   0.060  

# Le systéme M/M/1/L


- la Durée inter arrivés suit une lois expo de paramètre lmd >0
- la Durée de service suit une lois expo de paramètre U >0
- la source des clients est infinie 
- la taille de système est finie ( L ) 
- le nombre des serveurs est 1 
- la discipline de service est FIFO

In [107]:
from simpy import *
import random

maxClient= 100           # le nombre max des cleints pour la simulation 
L  = 2                   # Limite de la taille de la file  
maxTime = 8              # Limite de temps de simulation 
timeServ = 1/7.5         # Moyenne de durée de service 
arrivalMean = 1/5        # Moyenne de durée inter arrivés 
seed = 12345             # Seed pour la simulation 
k=0 


print ( "\nles caractéristiques de performance:\n ")

lmd = 1 / arrivalMean
U = 1 / timeServ
P = lmd / U
somme = 0 
for k in range(L+1) : 
    somme = somme+ P**k

P0 = 1/  somme   
PL =  ((1 - P) / (1- P**(L+1) ) ) * (P**L)

lmdE  = lmd*(1-PL)      #Le taux d'entrée moyen , il est différent de lambda car la taille de sys est limitée 

ns = (P/ (1-P)  ) - ( (L+1)* (P**(L+1))  /  (1-(P)**(L+1)  )  )
nf = ns  - P * (1-PL)
ts = ns/lmdE
tf = nf/lmdE

print("Intensité du traffic : %6.4f " %(P))
print("La probabilité d'avoir 0 clients dans le système P0 =  %6.4f" % ( P0))
print("La probabilité de refus , avoir L cleints dans le système PL =  %6.4f" % ( PL))
print("Nombre moyen de clients dans le système ns = %6.4f " %(ns))
print("Nombre moyen de clients dans la file nf= %6.4f " %(nf))
print("Temps de sejour moyen d’un client dans le systeme ts = %6.4f " %(ts))
print("Temps de séjour moyen d’un client dans la file tf = %6.4f  \n" %(tf))


# Setup and start the simulation
print("\nSimulation de système M/M/1/L ******************************************************************************************\n")
random.seed(seed)
env = Environment()
R =   Resource(env , capacity = 1 ) 
env.process(Source(env, maxClient, arrivalMean, R ,L ) )
env.run(until=maxTime)


les caractéristiques de performance:
 
Intensité du traffic : 0.6667 
La probabilité d'avoir 0 clients dans le système P0 =  0.4737
La probabilité de refus , avoir L cleints dans le système PL =  0.2105
Nombre moyen de clients dans le système ns = 0.7368 
Nombre moyen de clients dans la file nf= 0.2105 
Temps de sejour moyen d’un client dans le systeme ts = 0.1867 
Temps de séjour moyen d’un client dans la file tf = 0.0533  


Simulation de système M/M/1/L ******************************************************************************************

Heure :   0.000  ||  Client01: Arrivé ||  nombre des clients dans le système= [0] ||  nombre des clients dans la file= [0]
Heure :   0.000  ||  Client01: Libéré de la file , durée attente dans la file =  0.000 
Heure :   0.001  ||  Client01: Terminé , durée de service =  0.0014 
Heure :   0.108  ||  Client02: Arrivé ||  nombre des clients dans le système= [0] ||  nombre des clients dans la file= [0]
Heure :   0.108  ||  Client02: Libéré de la

# Le systéme M/M/s/L

- la Durée inter arrivés suit une lois expo de paramètre lmd >0
- la Durée de service suit une lois expo de paramètre U >0
- la source des clients est infinie 
- la taille de système est finie ( L ) 
- le nombre des serveurs est s 
- la discipline de service est FIFO

In [108]:
import math
from cmath import * 
maxClient= 20           # le nombre max des cleints pour la simulation 
maxTime = 4             # Limite de temps de simulation 
timeServ = 1/6          # Moyenne de durée de service 
arrivalMean = 1/9       # Moyenne de durée inter arrivés 
seed = 12345            # Seed pour la simulation 
s= 2                    # nombre des serveurs 
k=0 
L = 4

print("Simulation d'un systéme M/M/s/L  \n\n ")
print ( "\nles caractéristiques de performance:\n ")

lmd = 1 / arrivalMean
U = 1 / timeServ
P = lmd / (s*U)
somme = 0 
for k in range(s) : 
    somme = somme+  ((1/math.factorial(k)) * ((lmd/U)**k ) ) 

P0 = 1/ ( somme + ((1/math.factorial(s)) * ((lmd/U)**s) * (1-(P**(L-s+1)) /(1-P)   )   )  ) 
PL = (1/ (math.factorial(s)* s**(L-s)) ) * ((lmd/U ) ** L ) * P0

lmdE  = lmd*(1-PL)      #Le taux d'entrée moyen 
nf = (  (P* P0 * (s*P)**s ) / (math.factorial(s)* (1-P)**2)   ) * ( 1 - P**(L-s+1) - (1-P)*(L-s+1)* P**(L-s)  ) 

ns = nf + (lmdE / U )    
ts = ns/lmdE
tf = nf/lmdE


print("Intensité du traffic : %6.4f " %(P))
print("La probabilité d'avoir 0 clients dans le système P0 =  %6.4f" % ( P0))
print("La probabilité de refus d'un client =  %6.4f" % ( PL))
print("Nombre moyen de clients dans le système ns= %6.4f " %(ns))
print("Nombre moyen de clients dans la file nf= %6.4f " %(nf))
print("Temps de sejour moyen d’un client dans le systeme ts = %6.4f " %(ts))
print("Temps de séjour moyen d’un client dans la file tf = %6.4f  \n" %(tf))

print("\nSimulation de système M/M/s/L  ******************************************************************************************\n")
# Setup and start the simulation
random.seed(seed)
env = Environment()

env.process(Source(env, maxClient, arrivalMean, Resource(env,s) , L))
env.run(until=maxTime)

Simulation d'un systéme M/M/s/L  

 

les caractéristiques de performance:
 
Intensité du traffic : 0.7500 
La probabilité d'avoir 0 clients dans le système P0 =  0.5792
La probabilité de refus d'un client =  0.3665
Nombre moyen de clients dans le système ns= 2.1719 
Nombre moyen de clients dans la file nf= 1.2217 
Temps de sejour moyen d’un client dans le systeme ts = 0.3810 
Temps de séjour moyen d’un client dans la file tf = 0.2143  


Simulation de système M/M/s/L  ******************************************************************************************

Heure :   0.000  ||  Client01: Arrivé ||  nombre des clients dans le système= [0] ||  nombre des clients dans la file= [0]
Heure :   0.000  ||  Client01: Libéré de la file , durée attente dans la file =  0.000 
Heure :   0.002  ||  Client01: Terminé , durée de service =  0.0017 
Heure :   0.060  ||  Client02: Arrivé ||  nombre des clients dans le système= [0] ||  nombre des clients dans la file= [0]
Heure :   0.060  ||  Client02: 

# Le systéme M/M/s/s

- la Durée inters arrivés suit une lois expo de paramètre lmd >0
- la Durée de service suit une lois expo de paramètre U >0
- la source des clients est infinie 
- la taille de système est s 
- le nombre des serveurs est s 
- la discipline de service est FIFO

le client ne peut pas rentrer si tous les serveurs sont occupés , donc pas de file d'attente 

In [109]:
import math
maxClient= 20           # le nombre max des cleints pour la simulation 
maxTime = 4             # Limite de temps de simulation 
timeServ = 1/7.5          # Moyenne de durée de service 
arrivalMean = 1/7       # Moyenne de durée inter arrivés 
seed = 12345            # Seed pour la simulation 
s= 2                    # nombre des serveurs 
k=0 

print("Simulation d'un systéme M/M/s/L  \n\n ")
print ( "\nles caractéristiques de performance:\n ")

lmd = 1 / arrivalMean
U = 1 / timeServ
P = lmd / U
somme = 0 
for k in range(s) : 
    somme = somme +  ((1/math.factorial(k)) * ((lmd/U)**k ) ) 

    
P0 = 1 / somme 
Ps = (1/math.factorial(s) ) * ((lmd/U ) ** s ) * P0
lmdE  = lmd*(1-Ps)      #Le taux d'entrée moyen 
nf = 0
ns = P*(1-Ps)  
ts = 1/U
tf = 0


print("La probabilité d'avoir 0 clients dans le système à un instant t :  P0 =  %6.4f" % ( P0))
print("la probabilité pour que les s serveurs soient occupés Ps =   %6.4f" % ( Ps))
print("Nombre moyen de clients dans le système = %6.4f " %(ns))
print("Nombre moyen de clients dans la file = %6.4f " %(nf))
print("Temps de sejour moyen d’un client dans le systeme ts = %6.4f " %(ts))
print("Temps de séjour moyen d’un client dans la file tf = %6.4f  \n" %(tf))

print("\nSimulation de système M/M/s/s  ******************************************************************************************\n")
# Setup and start the simulation
random.seed(seed)
env = Environment()

env.process(Source(env, maxClient, arrivalMean, Resource(env,s) , s))
env.run(until=maxTime)

Simulation d'un systéme M/M/s/L  

 

les caractéristiques de performance:
 
La probabilité d'avoir 0 clients dans le système à un instant t :  P0 =  0.5172
la probabilité pour que les s serveurs soient occupés Ps =   0.2253
Nombre moyen de clients dans le système = 0.7231 
Nombre moyen de clients dans la file = 0.0000 
Temps de sejour moyen d’un client dans le systeme ts = 0.1333 
Temps de séjour moyen d’un client dans la file tf = 0.0000  


Simulation de système M/M/s/s  ******************************************************************************************

Heure :   0.000  ||  Client01: Arrivé ||  nombre des clients dans le système= [0] ||  nombre des clients dans la file= [0]
Heure :   0.000  ||  Client01: Libéré de la file , durée attente dans la file =  0.000 
Heure :   0.001  ||  Client01: Terminé , durée de service =  0.0014 
Heure :   0.077  ||  Client02: Arrivé ||  nombre des clients dans le système= [0] ||  nombre des clients dans la file= [0]
Heure :   0.077  ||  Clien

# Le systéme M/M/∞

- la Durée inter arrivés suit une lois expo de paramètre lmd >0
- la Durée de service suit une lois expo de paramètre U >0
- la source des clients est infinie 
- la taille de système est infinie  
- le nombre des serveurs est infini 
- la discipline de service est FIFO

C’est un système, comprenant une infinité de stations de service identiques. Dans ce cas pas de file d'attente , Chaque client est servi dès son arrive.

In [110]:
import math
maxClient= 100          # le nombre max des cleints pour la simulation 
maxTime = 4             # Limite de temps de simulation 
timeServ = 1/6          # Moyenne de durée de service 
arrivalMean = 1/9       # Moyenne de durée inter arrivée 
seed = 12345            # Seed pour la simulation 
s= 100000               # nombre des serveurs infini , posons s= 100000
L = -1                  # taille max = -1 pour dire qu'elle est infinie 


print ( "\nles caractéristiques de performance:\n ")

lmd = 1 / arrivalMean                                          
U = 1 / timeServ                                                                                            
P0 = exp(-lmd/U)
ns = lmd/ U
ts = 1/U



print("La probabilité  P0 =  %s" % ( P0))
print("Nombre moyen de clients dans le système ns = %6.4f " %(ns))
print("Temps de sejour moyen d’un client dans le systeme ts = %6.4f " %(ts))



# Setup and start the simulation
print("\nSimulation de système M/M/∞  ******************************************************************************************\n")
random.seed(seed)
env = Environment()
env.process(Source(env, maxClient, arrivalMean, Resource(env,s) , L))
env.run(until=maxTime)



les caractéristiques de performance:
 
La probabilité  P0 =  (0.22313016014842982+0j)
Nombre moyen de clients dans le système ns = 1.5000 
Temps de sejour moyen d’un client dans le systeme ts = 0.1667 

Simulation de système M/M/∞  ******************************************************************************************

Heure :   0.000  ||  Client01: Arrivé ||  nombre des clients dans le système= [0] ||  nombre des clients dans la file= [0]
Heure :   0.000  ||  Client01: Libéré de la file , durée attente dans la file =  0.000 
Heure :   0.002  ||  Client01: Terminé , durée de service =  0.0017 
Heure :   0.060  ||  Client02: Arrivé ||  nombre des clients dans le système= [0] ||  nombre des clients dans la file= [0]
Heure :   0.060  ||  Client02: Libéré de la file , durée attente dans la file =  0.000 
Heure :   0.119  ||  Client02: Terminé , durée de service =  0.0591 
Heure :   0.254  ||  Client03: Arrivé ||  nombre des clients dans le système= [0] ||  nombre des clients dans la fi